
# Laboratorium 2 - rozwiązanie
## Analiza danych - DataFrames
**Paweł Kruczkiewicz**

### Zadanie 
- Napisać program w języku Julia do obliczania iloczynu skalarnego wektorów (<a href="https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#LinearAlgebra.dot">LinearAlgebra.dot</a>) i mnożenia macierzy kwadratowej przez wektor z użyciem 
<a href="https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#Elementary-operations-1">operatora *</a>.
- Uruchomić i zmierzyć czasy działania obydwu funkcji - każdej dla różnych rozmiarów wektorów.  Dokonać 10 pomiarów dla każdego rozmiaru wektora.
- Czasy działania powinny  być zapisywane do <b>jednego</b> pliku CSV. Należy zaplanować odpowiednią strukturę kolumn takiego pliku.
- Wczytać dane z w/w pliku do <b>jednego</b> DataFrame w języku Julia. 
- Korzystająć z mechanizmów DataFrame w języku Julia obliczyć średnią i odchylenie standardowe, w taki sposób, aby narysować wykresy średnich czasów obliczenia operacji w zależności od rozmiaru wektora. Dodać słupki błędów do obydwu wykresów uzyskanych z obliczenia odchylenia standardowego. 
- Proszę poukładac wykresy  w tabelkę za pomocą  opcji layouts: https://docs.juliaplots.org/latest/layouts/#Simple-Layouts-1.
- Należy zadbać o staranne podpisanie osi i wykresów.

## Rozwiązanie
### Deklaracje funkcji

Przyjmują wymiary macierzy, zwracają informacje na temat czasu wykonania odpowiednich funkcji

In [1]:
using Plots
using DataFrames
using Statistics
using CSV

using LinearAlgebra
scalarProductTest(n::Int64) = (A = rand(Float64, n); B = rand(Float64, n); @timed LinearAlgebra.dot(A, B))
squareMatrixMultiplicationTest(n::Int64) = (A = rand(Float64, n, n); B = rand(Float64, n); @timed A*B)
#timed zwraca [<wynik>, <czas>, ...]

squareMatrixMultiplicationTest (generic function with 1 method)

### Stworzenie dataframe'u

In [ ]:
#n_s = map(x->10^x,(2:7))
n_s = 1000:1000:10000

size_of_vectors = Int[]
time_measure = Float32[]
measure_type = Int[]
for n = n_s
    for i = 1:10
        if n < 10^5
            push!(size_of_vectors, n)
            push!(time_measure, squareMatrixMultiplicationTest(n)[2])
            push!(measure_type, 1)
        end
        tmp_n = n*1000
        push!(size_of_vectors, tmp_n)
        push!(time_measure, scalarProductTest(tmp_n)[2])
        push!(measure_type, 0)

    end
end

measurements_df = DataFrame(Rozmiar = size_of_vectors,
    Czas = time_measure,
    Typ = measure_type)
show(measurements_df)

### Zapis do pliku csv

In [3]:
CSV.write("time_measurements.csv", measurements_df)

"time_measurements.csv"

### Wczytanie z powrotem

In [ ]:
input="time_measurements.csv"
mydata=CSV.read(input, delim=",",DataFrame)

### Zabawa z danymi

In [ ]:
combined_df = combine(groupby(mydata, [:Typ, :Rozmiar]), :Czas => mean, :Czas => std)

In [6]:
skalar_df = filter(row -> row[:Typ] == 0, combined_df)
multiply_df = filter(row -> row[:Typ] == 1, combined_df)

,Typ,Rozmiar,Czas_mean,Czas_std
,Int64,Int64,Float64,Float64
1,1,1000,0.00075507,0.000130624
2,1,2000,0.0026651,0.000229227
3,1,3000,0.005806,0.000561734
4,1,4000,0.00897754,0.000752277
5,1,5000,0.0146968,0.000394366
6,1,6000,0.0213052,0.000551326
7,1,7000,0.0289553,0.000997842
8,1,8000,0.0347673,0.000904235
9,1,9000,0.0472141,0.00199947


### Rysowanie

In [1]:
skalar = scatter(skalar_df.Rozmiar,
    skalar_df.Czas_mean,
    colour = :red,
    yerr=skalar_df.Czas_std,
    title= "Iloczyn Skalarny")
mnozenie = scatter(multiply_df.Rozmiar,
    multiply_df.Czas_mean,
    colour = :blue,
    yerr=skalar_df.Czas_std,
    title= "Mnożenie macierzy")
tablica = [skalar, mnozenie]
plot(tablica, layout=2, ylabel = "Czas [s]", xlabel= "Wielkość elementów")

LoadError: [91mUndefVarError: skalar_df not defined[39m